# SIR, birth and death 
{func}`.SIR_Birth_Death`

Here we consider an SIR model in which individuals may be removed by death from each compartment at a uniform rate per person, $\gamma$.
The population is replenished via births into the susceptible compartment at the same rate, thus conserving the total population by design.
For deterministic evolution, the population size remains constant whereas for stochastic evolution, the size fluctuates around this value.
The equations are as follows:

$$\begin{aligned}
\frac{\mathrm{d}S}{\mathrm{d}t} &= \mu N - \frac{\beta SI}{N} - \mu S \\
\frac{\mathrm{d}I}{\mathrm{d}t} &= \frac{\beta SI}{N} - \gamma I - \mu I \\
\frac{\mathrm{d}R}{\mathrm{d}t} &= \gamma I - \mu R
\end{aligned}$$

As an example, we study stochastic evolution of this system with measles-like parameters in 3 differently sized populations.
This provides a demonstration of threshold population sizes in order to support endemic circulation of certain pathogens.

In [ ]:
from pygom import common_models
import matplotlib.pyplot as plt
import numpy as np
import math

#####################
# Set up PyGOM object
#####################

# Parameters
n_pop=1e4
mu=0.01/365    # birth/death rate 1% per year
gamma=1/20 
R0=15
beta=R0*gamma

ode = common_models.SIR_Birth_Death({'beta':beta, 'gamma':gamma, 'mu':mu, 'N':n_pop})

# Time range and increments
tmax=365*10                           # maximum time over which to run solver
dt=1                                  # timestep
n_timestep=math.ceil(tmax/dt)         # number of iterations
t = np.linspace(0, tmax, n_timestep)  # times at which solution will be evaluated

# Initial conditions (endemic equilibrium derived from stationary point)
def sir_bd_endemic_eq(mu, beta, gamma, n_pop):
    s0=math.floor((gamma+mu)*n_pop/beta)
    i0=math.floor(mu*(n_pop-s0)*n_pop/(beta*s0))
    r0=n_pop-(s0+i0)
    return [s0, i0, r0]

x0=sir_bd_endemic_eq(mu, beta, gamma, n_pop)
ode.initial_values = (x0, t[0])

##########
# Simulate
##########
n_sim=10
np.random.seed(1)

solution, simT = ode.solve_stochast(t, n_sim, full_output=True)
y=np.dstack(solution)

############################
# try larger population size
############################
n_pop=1e5
ode = common_models.SIR_Birth_Death({'beta':beta, 'gamma':gamma, 'mu':mu, 'N':n_pop})  # update parameter
x0=sir_bd_endemic_eq(mu, beta, gamma, n_pop)  # recalculate IC's
ode.initial_values = (x0, t[0])
solution_2, simT_2 = ode.solve_stochast(t, n_sim, full_output=True)  # simulate
y_2=np.dstack(solution_2)

#################################
# try even larger population size
#################################
n_pop=1e6
ode = common_models.SIR_Birth_Death({'beta':beta, 'gamma':gamma, 'mu':mu, 'N':n_pop})  # update parameter
x0=sir_bd_endemic_eq(mu, beta, gamma, n_pop)  # recalculate IC's
ode.initial_values = (x0, t[0])
solution_3, simT_3 = ode.solve_stochast(t, n_sim, full_output=True)  # simulate
y_3=np.dstack(solution_3)

Plotting the results, we see that for populations of sizes 10,000 and 100,000, the infected population is critically close to zero, such that stochastic fluctuations eventually lead to disease extinction.
This is of course signified by the infected class reaching zero, but also by the recovered and susceptible classes undergoing stable linear growth due to population turnover.
When the population size is 1,000,000, we see that the infected subset, of typical size 500, is able to persist for the full 10 years of the simulation.

In [ ]:
f, axarr = plt.subplots(3,3, layout='constrained', figsize=(10, 5))

for i in range(0,3):
    # Plot individual trajectories
    for j in range(0, n_sim):
        axarr[0][i].plot(t/365, y[:,i,j], alpha=0.4, color="C0")
        axarr[1][i].plot(t/365, y_2[:,i,j], alpha=0.4, color="C1")
        axarr[2][i].plot(t/365, y_3[:,i,j], alpha=0.4, color="C2")

# Add titles
stateList = ['S', 'I', 'R']
for idx, state in enumerate(stateList):
    axarr[0][idx].set_ylabel(state, rotation=0)
    axarr[1][idx].set_ylabel(state, rotation=0)
    axarr[2][idx].set_ylabel(state, rotation=0)
    axarr[0][idx].set_xlabel('Time (years)')
    axarr[1][idx].set_xlabel('Time (years)')
    axarr[2][idx].set_xlabel('Time (years)')

axarr[0][1].set_title("Population size = 10,000")
axarr[1][1].set_title("Population size = 100,000")
axarr[2][1].set_title("Population size = 1,000,000")

plt.show()